In [ ]:
context = '[DOC] [TLE] J! Archive - Show #5243, aired 2007-05-30 [PAR] May 30, 2007 ... 1976: "A Single Colorado Mountain". $1200, 19. Founded in the 1530s, this \ncapital of Jalisco state is the second-largest in Mexico. $1200, 23... [DOC] [TLE] November 14, 2008 [PAR] ... "Regular Folks" Ordinary People 1932: "Magnificent Inn" Grand Hotel 1976: "A \nSingle Colorado Mountain" Rocky Down Mexico Way: In 1986 Mexico scored... [DOC] [TLE] LAND USE AND LANDSCAPE CHANGE IN THE COLORADO ... [PAR] ... landscape change in a single Colorado mountain valley using methods from \n..... and for multi-family 1970 1973 1976 1979 1982 1985 1988 1991 complexes... [DOC] [TLE] J! set #1 Flashcards | Quizlet [PAR] 18 shows included Learn with flashcards, games, and more  for free. '

from errudite.processor import SpacyAnnotator
spacy_annotator = SpacyAnnotator(disable=['parser', 'ner', 'textcat'])
list(spacy_annotator.process_text(context))

In [ ]:
import glob, os

import errudite
from errudite.io import DatasetReader
from errudite.predictors import Predictor
from errudite.targets.instance import Instance
from errudite.targets.label import Label
from errudite.utils import accuracy_score, normalize_file_path

DATASET_FOLDER = normalize_file_path("~/datasets/raw_data/mrqa/out_of_domain_devs")
sample_size=100000
file_path = ",".join(glob.glob(os.path.join(DATASET_FOLDER, "*.jsonl.gz")))
reader = DatasetReader.by_name("mrqa")(
    cache_folder_path=f"~/datasets/caches/dataset_debug/mrqa-{sample_size}")
instances = reader.read(file_path, sample_size=sample_size)

In [2]:
%load_ext autoreload
%autoreload 2
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

import matplotlib
import numpy as np
import matplotlib.pyplot as plt
import os
def import_sys():
    import sys
    sys.path.append('..')
    sys.path.append('../..')
import_sys()

import logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)  # pylint: disable=invalidzx-name

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
import glob, os

import errudite
from errudite.io import DatasetReader
from errudite.predictors import Predictor
from errudite.targets.instance import Instance
from errudite.targets.label import Label
from errudite.utils import accuracy_score, normalize_file_path

DATASET_FOLDER = normalize_file_path("~/datasets/raw_data/mrqa/in_domain_devs")
sample_size=10
file_path = ",".join(glob.glob(os.path.join(DATASET_FOLDER, "*.jsonl.gz")))
reader = DatasetReader.by_name("mrqa")(
    cache_folder_path=f"~/datasets/caches/dataset_debug/mrqa-{sample_size}")
instances = reader.read(file_path, sample_size=sample_size)

MODEL_FOLDER = normalize_file_path("~/datasets/models/mrqa/")
predictor = Predictor.by_name("mrqa")(
    name="mrqa_path", 
    model_path=os.path.join(MODEL_FOLDER, "mrqa_bert_base.gz"))

INFO:errudite.utils.file_utils:Errudite cache folder selected: /Users/tongshuangwu/datasets/caches/dataset_debug/mrqa-10
INFO:errudite.io.dataset_reader:Reading instances from lines in file at: /Users/tongshuangwu/datasets/raw_data/mrqa/in_domain_devs/NaturalQuestions.jsonl.gz
INFO:errudite.io.mrqa_reader:Reading from dataset: NaturalQuestionsShort (in).
8it [00:00, 34.42it/s]
INFO:allennlp.models.archival:loading archive file /Users/tongshuangwu/datasets/models/mrqa/mrqa_bert_base.gz
INFO:allennlp.models.archival:extracting archive file /Users/tongshuangwu/datasets/models/mrqa/mrqa_bert_base.gz to temp dir /var/folders/9k/yx4ryhp918qfxx59mmp174lh0000gn/T/tmpftogm90q
INFO:allennlp.common.params:type = default
INFO:allennlp.data.vocabulary:Loading token dictionary from /var/folders/9k/yx4ryhp918qfxx59mmp174lh0000gn/T/tmpftogm90q/vocabulary.
INFO:allennlp.common.from_params:instantiating class <class 'allennlp.models.model.Model'> from params {'initializer': [], 'text_field_embedder': {'

In [7]:
instances[0].question.metas

{'dataset': 'NaturalQuestionsShort', 'domain': 'in'}

In [3]:
import pandas as pd
from tqdm import tqdm
predictors = { p.name: p for p in [predictor] }
predictions = { p: [] for p in predictors }
logger.info("Running predictions....")
for instance in tqdm(instances):
    instance_predictions = []
    for predictor in predictors.values():
        prediction = Predictor.by_name("qa_task_class").model_predict(
            predictor, 
            instance.question, 
            instance.context, 
            instance.groundtruths)
        instance_predictions.append(prediction)
        #print(prediction.doc, prediction.span_start, prediction.span_end, instance.context)
        predictions[predictor.name].append(prediction)
    instance.set_entries(predictions=instance_predictions)
for predictor in predictors.values():
    predictor.evaluate_performance(instances)
print(pd.DataFrame([ {"predictor": p.name, "f1": p.perform["f1"] } for p in predictors.values() ]))
Instance.build_instance_hashes(instances)

INFO:__main__:Running predictions....
 10%|█         | 1/10 [00:12<01:53, 12.64s/it]

[{'best_span_str': 'acid', 'qid': 'tmp', 'best_span_logit': 8.56551742553711, 'char_offsets': [2476, 2480]}, {'best_span_str': 'acid', 'qid': 'tmp', 'best_span_logit': 8.56551742553711, 'char_offsets': [2476, 2480]}, {'best_span_str': 'acid', 'qid': 'tmp', 'best_span_logit': 8.56551742553711, 'char_offsets': [2476, 2480]}, {'best_span_str': 'acid', 'qid': 'tmp', 'best_span_logit': 8.56551742553711, 'char_offsets': [2476, 2480]}]


 20%|██        | 2/10 [00:27<01:46, 13.29s/it]

[{'best_span_str': '1927', 'qid': 'tmp', 'best_span_logit': 10.251514434814453, 'char_offsets': [26, 30]}, {'best_span_str': '1927', 'qid': 'tmp', 'best_span_logit': 10.251514434814453, 'char_offsets': [26, 30]}, {'best_span_str': '1927', 'qid': 'tmp', 'best_span_logit': 10.251514434814453, 'char_offsets': [26, 30]}, {'best_span_str': '1927', 'qid': 'tmp', 'best_span_logit': 10.251514434814453, 'char_offsets': [26, 30]}]


 30%|███       | 3/10 [00:42<01:35, 13.70s/it]

...

 90%|█████████ | 9/10 [02:02<00:12, 12.86s/it]

[{'best_span_str': 'Sir Pelinore', 'qid': 'tmp', 'best_span_logit': 8.468249320983887, 'char_offsets': [1464, 1476]}, {'best_span_str': 'Sir Pelinore', 'qid': 'tmp', 'best_span_logit': 8.468249320983887, 'char_offsets': [1464, 1476]}, {'best_span_str': 'Sir Pelinore', 'qid': 'tmp', 'best_span_logit': 8.468249320983887, 'char_offsets': [1464, 1476]}, {'best_span_str': 'Sir Pelinore', 'qid': 'tmp', 'best_span_logit': 8.468249320983887, 'char_offsets': [1464, 1476]}]


100%|██████████| 10/10 [02:13<00:00, 12.25s/it]

[{'best_span_str': 'London town', 'qid': 'tmp', 'best_span_logit': 7.203769683837891, 'char_offsets': [221, 232]}, {'best_span_str': 'London town', 'qid': 'tmp', 'best_span_logit': 7.203769683837891, 'char_offsets': [221, 232]}, {'best_span_str': 'London town', 'qid': 'tmp', 'best_span_logit': 7.203769683837891, 'char_offsets': [221, 232]}, {'best_span_str': 'London town', 'qid': 'tmp', 'best_span_logit': 7.203769683837891, 'char_offsets': [221, 232]}]
         f1  predictor
0  0.557143  mrqa_path


({InstanceKey(qid='d94a42693350473581ff79dc91c91e04', vid=0): Instance [InstanceKey(qid='d94a42693350473581ff79dc91c91e04', vid=0)],
  InstanceKey(qid='f41dbe24bed44870a8ad36c87dda59a2', vid=0): Instance [InstanceKey(qid='f41dbe24bed44870a8ad36c87dda59a2', vid=0)],
  InstanceKey(qid='34f8093a16b64c8097bdaa03cccdef37', vid=0): Instance [InstanceKey(qid='34f8093a16b64c8097bdaa03cccdef37', vid=0)],
  InstanceKey(qid='b6a13cac6289435697e8ff98d55854a9', vid=0): Instance [InstanceKey(qid='b6a13cac6289435697e8ff98d55854a9', vid=0)],
  InstanceKey(qid='52ffe790d24a41669c08a240c1d45114', vid=0): Instance [InstanceKey(qid='52ffe790d24a41669c08a240c1d45114', vid=0)],
  InstanceKey(qid='d0d647812b8d47e88327e05abd34c492', vid=0): Instance [InstanceKey(qid='d0d647812b8d47e88327e05abd34c492', vid=0)],
  InstanceKey(qid='5e54f1fb51b34d5c91636ca9a1d3518a', vid=0): Instance [InstanceKey(qid='5e54f1fb51b34d5c91636ca9a1d3518a', vid=0)],
  InstanceKey(qid='ee477db3268c4a0b8321e3cb971573a7', vid=0): Instanc

In [4]:
for predictor in predictors.values():
    predictor.evaluate_performance(instances)
print(pd.DataFrame([ {"predictor": p.name, "f1": p.perform["f1"] } for p in predictors.values() ]))

         f1  predictor
0  0.557143  mrqa_path


[autoreload of errudite.predictors.qa.mrqa_allennlp.mrqa_predictor failed: Traceback (most recent call last):
  File "/Users/tongshuangwu/sourcetree/errudite_dataset_debug/venv/lib/python3.6/site-packages/IPython/extensions/autoreload.py", line 245, in check
    superreload(m, reload, self.old_objects)
  File "/Users/tongshuangwu/sourcetree/errudite_dataset_debug/venv/lib/python3.6/site-packages/IPython/extensions/autoreload.py", line 434, in superreload
    module = reload(module)
  File "/Users/tongshuangwu/sourcetree/errudite_dataset_debug/venv/lib/python3.6/imp.py", line 315, in reload
    return importlib.reload(module)
  File "/Users/tongshuangwu/sourcetree/errudite_dataset_debug/venv/lib/python3.6/importlib/__init__.py", line 166, in reload
    _bootstrap._exec(spec, module)
  File "<frozen importlib._bootstrap>", line 618, in _exec
  File "<frozen importlib._bootstrap_external>", line 678, in exec_module
  File "<frozen importlib._bootstrap>", line 219, in _call_with_frames_rem

In [ ]:
import pandas as pd
pd.DataFrame([ {"predictor": p.name, "f1": p.perform["f1"] } for p in predictors.values() ])

In [ ]:
TRAIN_DATASET_FOLDER = normalize_file_path("~/datasets/raw_data/mrqa/trains")
sample_size=100
file_path = ",".join(glob.glob(os.path.join(DATASET_FOLDER, "*.jsonl.gz"))[:2])
reader.count_vocab_freq(file_path)